
# Soft Computing PG (7197)
## Assignment 1



Ajul Thomas (*u3253992@uni.canberra.edu.au*)

#### Using neural networks to predictr the velocity of robots

1. Import required libraries
2. load data
3. Data exploration and visualisatoion
4. Feature engineering and preprocessing
5. Model Selection
6. Model Evaluation
7. Final Model Selection





In [2]:
# import required libraries
import pandas as pd




In [5]:
# setting colnames as the column names of the dataframe
headers = [
    'X_robot', 'Y_robot', 'Orientation_robot', 'Collision', 
    'X_candle1', 'Y_candle1', 'X_candle2', 'Y_candle2', 
    'X_candle3', 'Y_candle3', 'X_candle4', 'Y_candle4', 
    'X_speed', 'Y_speed'
]


# loading data from the csv file
robots_df = pd.read_csv('./data/RobotData.csv', names=headers)

# displaying the first 5 rows of the dataframe
robots_df.head()



,X_robot,Y_robot,Orientation_robot,Collision,X_candle1,Y_candle1,X_candle2,Y_candle2,X_candle3,Y_candle3,X_candle4,Y_candle4,X_speed,Y_speed
0,3.955138,11.870423,-2.708559,0,1.566351,10.681844,2.69308,6.687333,0.187108,1.678316,8.4906,9.150135,-0.118001,-0.054551
1,4.000958,4.238834,1.812858,0,1.566351,10.681844,2.69308,6.687333,0.187108,1.678316,8.4906,9.150135,-0.031162,0.126210
2,8.755436,2.019115,-1.135413,0,1.566351,10.681844,2.69308,6.687333,0.187108,1.678316,8.4906,9.150135,0.054829,-0.117872
3,11.098697,0.086990,0.044513,0,1.566351,10.681844,2.69308,6.687333,0.187108,1.678316,8.4906,9.150135,0.129871,0.005785
4,10.049046,5.131111,-1.354733,0,1.566351,10.681844,2.69308,6.687333,0.187108,1.678316,8.4906,9.150135,0.027870,-0.126977


In [6]:
# shape of the dataframe
print(robots_df.shape)

(19998, 14)


In [7]:
# exploring the dataset
robots_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19998 entries, 0 to 19997
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   X_robot            19998 non-null  float64
 1   Y_robot            19998 non-null  float64
 2   Orientation_robot  19998 non-null  float64
 3   Collision          19998 non-null  int64  
 4   X_candle1          19998 non-null  float64
 5   Y_candle1          19998 non-null  float64
 6   X_candle2          19998 non-null  float64
 7   Y_candle2          19998 non-null  float64
 8   X_candle3          19998 non-null  float64
 9   Y_candle3          19998 non-null  float64
 10  X_candle4          19998 non-null  float64
 11  Y_candle4          19998 non-null  float64
 12  X_speed            19998 non-null  float64
 13  Y_speed            19998 non-null  float64
dtypes: float64(13), int64(1)
memory usage: 2.1 MB


In [8]:
# having a quick look at the data stats

robots_df.describe()

,X_robot,Y_robot,Orientation_robot,Collision,X_candle1,Y_candle1,X_candle2,Y_candle2,X_candle3,Y_candle3,X_candle4,Y_candle4,X_speed,Y_speed
count,19998.000000,19998.000000,19998.000000,19998.000000,19998.000000,19998.000000,19998.000000,19998.000000,19998.000000,19998.000000,19998.000000,19998.000000,19998.000000,19998.000000
mean,4.889966,5.008106,-0.122435,0.107961,5.199080,5.312078,4.826039,4.901476,5.378227,5.031117,5.011059,5.264268,-0.001080,-0.001360
std,3.147421,3.136672,2.315616,0.310339,3.208729,3.274560,3.224860,3.167517,3.185632,2.899557,3.044050,3.104862,0.091996,0.091840
min,0.000000,0.000000,-8.621953,0.000000,0.025638,0.028771,0.027746,0.028143,0.118247,0.109839,0.007778,0.016444,-0.130000,-0.130000
25%,2.233069,2.464950,-1.761002,0.000000,2.542015,2.676122,2.074969,2.143751,3.032612,2.389895,2.284132,2.763607,-0.093390,-0.094059
50%,4.665229,4.921910,-0.043931,0.000000,5.242550,4.989841,4.539118,4.744344,5.235521,5.069661,5.216678,5.218924,-0.000343,-0.002989
75%,7.072524,7.208669,1.623784,0.000000,7.632224,7.546125,6.781621,7.131440,7.609577,7.069442,6.814157,7.587753,0.090322,0.090405
max,12.000000,12.000000,9.792638,1.000000,11.988487,11.994296,11.999628,11.884932,11.935167,11.960463,11.911963,11.596468,0.130000,0.130000


#### Feature Engineering

- calculate the distance between robot and candles
- Standardise the features

In [9]:
# function to calculate euclidean distance between two points
def euclidean_distance(x1, y1, x2, y2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

# calculating the distance between the robot and each candle
robots_df['distance_candle1'] = euclidean_distance(robots_df['X_robot'], robots_df['Y_robot'], robots_df['X_candle1'], robots_df['Y_candle1'])
robots_df['distance_candle2'] = euclidean_distance(robots_df['X_robot'], robots_df['Y_robot'], robots_df['X_candle2'], robots_df['Y_candle2'])
robots_df['distance_candle3'] = euclidean_distance(robots_df['X_robot'], robots_df['Y_robot'], robots_df['X_candle3'], robots_df['Y_candle3'])
robots_df['distance_candle4'] = euclidean_distance(robots_df['X_robot'], robots_df['Y_robot'], robots_df['X_candle4'], robots_df['Y_candle4'])

# displaying the first 5 rows of the dataframe
robots_df.head()

,X_robot,Y_robot,Orientation_robot,Collision,X_candle1,Y_candle1,X_candle2,Y_candle2,X_candle3,Y_candle3,X_candle4,Y_candle4,X_speed,Y_speed,distance_candle1,distance_candle2,distance_candle3,distance_candle4
0,3.955138,11.870423,-2.708559,0,1.566351,10.681844,2.69308,6.687333,0.187108,1.678316,8.4906,9.150135,-0.118001,-0.054551,2.668150,5.334531,10.866329,5.288703
1,4.000958,4.238834,1.812858,0,1.566351,10.681844,2.69308,6.687333,0.187108,1.678316,8.4906,9.150135,-0.031162,0.126210,6.887648,2.775912,4.593660,6.654154
2,8.755436,2.019115,-1.135413,0,1.566351,10.681844,2.69308,6.687333,0.187108,1.678316,8.4906,9.150135,0.054829,-0.117872,11.257256,7.651432,8.575104,7.135936
3,11.098697,0.086990,0.044513,0,1.566351,10.681844,2.69308,6.687333,0.187108,1.678316,8.4906,9.150135,0.129871,0.005785,14.251897,10.687325,11.027017,9.430947
4,10.049046,5.131111,-1.354733,0,1.566351,10.681844,2.69308,6.687333,0.187108,1.678316,8.4906,9.150135,0.027870,-0.126977,10.137394,7.518781,10.448906,4.310604


In [12]:
# df with only the distance, orientation, collision and speed columns
df_modified = robots_df[['distance_candle1', 'distance_candle2', 'distance_candle3', 'distance_candle4', 'Orientation_robot', 'Collision', 'X_speed', 'Y_speed']]
df_modified.head()


,distance_candle1,distance_candle2,distance_candle3,distance_candle4,Orientation_robot,Collision,X_speed,Y_speed
0,2.668150,5.334531,10.866329,5.288703,-2.708559,0,-0.118001,-0.054551
1,6.887648,2.775912,4.593660,6.654154,1.812858,0,-0.031162,0.126210
2,11.257256,7.651432,8.575104,7.135936,-1.135413,0,0.054829,-0.117872
3,14.251897,10.687325,11.027017,9.430947,0.044513,0,0.129871,0.005785
4,10.137394,7.518781,10.448906,4.310604,-1.354733,0,0.027870,-0.126977


In [13]:
# get the predictor and target variables
X = df_modified.drop(columns=['X_speed', 'Y_speed'])
y = df_modified[['X_speed', 'Y_speed']]

# check shape of the predictor and target variables
X.shape, y.shape

((19998, 6), (19998, 2))

In [14]:
# split the data set into train, test and validation sets
from sklearn.model_selection import train_test_split

X_train_validate, X_test, y_train_validate, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_validate, y_train, y_validate = train_test_split(X_train_validate, y_train_validate, test_size=0.125, random_state=42)

# check the shape of the train, test and validation sets
X_train.shape, X_test.shape, X_validate.shape


((13998, 6), (4000, 6), (2000, 6))

In [23]:
# scaling the data using the standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

array([[-0.92003981,  0.33949612, -1.06094074, -1.54649969,  0.24239072,
         2.8865204 ],
       [ 2.44487253,  1.33339215,  1.13603377,  3.06330734, -0.55316183,
         2.8865204 ],
       [-1.08940651, -0.42198615,  0.99583994,  0.55108554,  0.99467723,
        -0.34643788],
       ...,
       [-0.0393121 ,  0.70445624, -0.27331986, -1.55562191,  1.14230023,
         2.8865204 ],
       [-0.84057732,  0.17144049, -0.74120366, -0.18670402,  0.63409329,
        -0.34643788],
       [ 0.67649701, -1.32039434,  1.3065462 , -0.03896993, -2.43535107,
        -0.34643788]])

In [20]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_validate_scaled = pd.DataFrame(X_validate_scaled, columns=X_validate.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [21]:
X_train_scaled.head()

# with the values in non-scientific notation
X_train_scaled.describe()

,distance_candle1,distance_candle2,distance_candle3,distance_candle4,Orientation_robot,Collision
count,1.399800e+04,1.399800e+04,1.399800e+04,1.399800e+04,1.399800e+04,1.399800e+04
mean,-1.959348e-16,-3.939000e-16,-2.984706e-16,1.142107e-16,5.076030e-19,3.400940e-17
std,1.000036e+00,1.000036e+00,1.000036e+00,1.000036e+00,1.000036e+00,1.000036e+00
min,-1.596271e+00,-1.536616e+00,-1.608994e+00,-1.572054e+00,-3.621845e+00,-3.464379e-01
25%,-7.161850e-01,-7.187214e-01,-6.946719e-01,-6.861359e-01,-7.075290e-01,-3.464379e-01
50%,-1.506746e-02,-1.687632e-02,-2.495718e-02,-2.633007e-02,2.823442e-02,-3.464379e-01
75%,6.527661e-01,6.338998e-01,6.449653e-01,6.552228e-01,7.572184e-01,-3.464379e-01
max,3.372642e+00,3.394171e+00,3.247353e+00,3.545812e+00,4.298386e+00,2.886520e+00


In [24]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

def train_model(id, hyper_params, train_X, test_x, val_X):
    # train the model
    mlp_regressor = MLPRegressor(**hyper_params).fit(train_X, y_train)
    # predict the target on the train, validation and test sets
    y_train_pred = mlp_regressor.predict(train_X)
    y_val_pred = mlp_regressor.predict(val_X)
    y_test_pred = mlp_regressor.predict(test_x)
    # calculate the mean squared error on the train and validation sets
    train_score = mean_squared_error(y_train, y_train_pred)
    val_score = mean_squared_error(y_validate , y_val_pred)
    test_score = mean_squared_error(y_test, y_test_pred)
    return {
        'id': id,
        'hidden_layers': mlp_regressor.n_layers_ - 2,
        'architecture': mlp_regressor.hidden_layer_sizes,
        'activation': mlp_regressor.activation,
        'batch_size': mlp_regressor.batch_size,
        'solver':   mlp_regressor.solver,
        'learning_rate_init': mlp_regressor.learning_rate_init,
        'learning_rate':    mlp_regressor.learning_rate,
        'iterations': mlp_regressor.n_iter_,
        'train_score': train_score,
        'val_score': val_score,
        'test_score': test_score,
    }

In [70]:
# hyper params
hyper_param_options = [
    { 'verbose': False, 'random_state': 33, 'activation': 'relu', 'hidden_layer_sizes': (12, 24, 48, 96 ), 'learning_rate_init': 0.001, 'batch_size': 20},
    # { 'verbose': False, 'random_state': 33, 'activation': 'tanh'},
    # # TRY USING LOGISTIC SIGMOID ACTIVATION FUNCTION
    # { 'verbose': False, 'random_state': 33, 'activation': 'logistic'},
    # # TRY USING IDENTITY
    # { 'verbose': False, 'random_state': 33, 'activation': 'identity'},
]

 
observations_df = pd.DataFrame()


for index,hyper_param_option in enumerate(hyper_param_options):
    print(f"Training model {index}, with hyper params: {hyper_param_option}")
    result = train_model(index, hyper_param_option, X_train_scaled, X_test_scaled, X_validate_scaled)
    result_df = pd.DataFrame([result])
    observations_df = pd.concat([observations_df, result_df], ignore_index=True)  
    print(f"Training model {index}, with hyper params: {hyper_param_option} completed")
    # print("Test results: ", observations_df.iloc[index])

# observations_df.to_csv('./data/observations_task2_final2.csv', index=False)

observations_df

Training model 0, with hyper params: {'verbose': False, 'random_state': 33, 'activation': 'relu', 'hidden_layer_sizes': (12, 24, 48, 96), 'learning_rate_init': 0.001, 'batch_size': 20}
Training model 0, with hyper params: {'verbose': False, 'random_state': 33, 'activation': 'relu', 'hidden_layer_sizes': (12, 24, 48, 96), 'learning_rate_init': 0.001, 'batch_size': 20} completed


,id,hidden_layers,architecture,activation,batch_size,solver,learning_rate_init,learning_rate,iterations,train_score,val_score,test_score
0,0,4,"(12, 24, 48, 96)",relu,20,adam,0.001,constant,15,0.004848,0.004893,0.004999
